# Exploratory Data Analysis

## 1. Load the data

In [2]:
import pandas as pd

table_df = pd.read_csv('data/1-JanTweets.csv')
table_df.head()

,Unnamed: 0,date,language,likes,retweets,screen_name,tweet,tweet_id
0,0.0,2020-01-01 19:08:15,und,0.0,0.0,Lil_COVID,@rammthagreat 🤣😭😭,1.212450480137392e+18
1,1.0,2020-01-01 17:24:25,en,1.0,0.0,MattFrieman,@Crof @BioAndBaseball Anyone know if there is ...,1.2124243521694147e+18
2,2.0,2020-01-01 16:41:21,und,1.0,0.0,CoVid_19_0290,https://t.co/vzXNs2RAPR,1.2124135119655854e+18
3,3.0,2020-01-01 22:04:34,und,1.0,0.0,captmakret,@RJPilkenton @brianstelter https://t.co/oHZFv8...,1.2124948507650212e+18
4,4.0,2020-01-01 14:05:11,en,21.0,5.0,COVID_19_ZA,Today at King Dinizulu Hospital welcoming the ...,1.212374210057851e+18


In [3]:
# Loop through the whole list and remove the coloumn headings,
# since they repeat after every 100 entries

for i in range(len(table_df)):
    if table_df.loc[i, "tweet"] == "tweet":
        #print(i)
        table_df=table_df.drop(index=i)

In [4]:
print(table_df.head())


   Unnamed: 0                 date language likes retweets    screen_name  \
0         0.0  2020-01-01 19:08:15      und   0.0      0.0      Lil_COVID   
1         1.0  2020-01-01 17:24:25       en   1.0      0.0    MattFrieman   
2         2.0  2020-01-01 16:41:21      und   1.0      0.0  CoVid_19_0290   
3         3.0  2020-01-01 22:04:34      und   1.0      0.0     captmakret   
4         4.0  2020-01-01 14:05:11       en  21.0      5.0    COVID_19_ZA   

                                               tweet                tweet_id  
0                                  @rammthagreat 🤣😭😭   1.212450480137392e+18  
1  @Crof @BioAndBaseball Anyone know if there is ...  1.2124243521694147e+18  
2                            https://t.co/vzXNs2RAPR  1.2124135119655854e+18  
3  @RJPilkenton @brianstelter https://t.co/oHZFv8...  1.2124948507650212e+18  
4  Today at King Dinizulu Hospital welcoming the ...   1.212374210057851e+18  


## 2. Removing '@' mentions
@ mentions in the tweets aren't really useful.
We can remove them from all the tweets if any tweet
mentions another user.


In [5]:
# Get the row data from the tuple returned by iterarrows()
next(table_df.iterrows())[1]

Unnamed: 0                         0
date             2020-01-01 19:08:15
language                         und
likes                            0.0
retweets                         0.0
screen_name                Lil_COVID
tweet              @rammthagreat 🤣😭😭
tweet_id       1.212450480137392e+18
Name: 0, dtype: object

In [6]:
# Check for presence of @ character
table_df["tweet"].str.contains("@")

0         True
1         True
2        False
3         True
4        False
         ...  
23795    False
23796    False
23797    False
23798    False
23799     True
Name: tweet, Length: 23565, dtype: bool

In [7]:
# Small example showing how to remove @ mentions
import re
text="@archity how are you?"
newText=re.sub('@.*? ', '', text)
print(newText)

how are you?


In [8]:
df_clean=table_df

# Function for removing @ mentions
def remove_mentions(text):
    newtext=re.sub('@.*? ', '', text)
    return newtext

df_clean['tweet']=table_df.tweet.apply(lambda x: remove_mentions(x))

print(df_clean.head(20)["tweet"])

# Get the data type of each coloumn
#print(df_clean.dtypes)

0                                                   🤣😭😭
1     Anyone know if there is a specific word for SA...
2                               https://t.co/vzXNs2RAPR
3                               https://t.co/oHZFv83UgZ
4     Today at King Dinizulu Hospital welcoming the ...
5                               https://t.co/lHpiq8F1vo
6     #فكرة ..\n#استخدمها بدلا من الزمور ❤️\n\nحط ال...
7                              What’s a good food spot?
8           Lmaoo damn what a way to start the new year
9     Thank you man! so glad we get to live another ...
10    Osea alguien te jala y debes aguantar. No seas...
11                                            Nasty? 🥴🤣
12    Bizle ne alakası var aq beni niye etiketliyon ...
13    My 2 year old nephew walks around the house an...
14    Earth: Makes a full rotation around the sun\nH...
15    Lol, yep I'm lifting now. It's like a morgue i...
16             Love these pups. https://t.co/nxaqEDK76u
17    plying to @ironorehopper\nInteresting, fur

In [38]:
# Go through each tweet and put individual word into a list
from collections import Counter

print("Coloumn tweet type: ", df_clean.tweet.dtypes)

word_list = []

for tweet in df_clean.tweet:
    word_list+=(tweet.split())


#word_list[:100]
print(Counter(word_list).most_common(20))

print(len(word_list), "total words")

Coloumn tweet type:  object
[('coronavirus', 7943), ('the', 7341), ('de', 7002), ('a', 6008), ('of', 5632), ('to', 5190), ('in', 5181), ('China', 3948), ('#coronavirus', 3344), ('en', 3174), ('Coronavirus', 3040), ('el', 2913), ('que', 2866), ('and', 2830), ('is', 2440), ('la', 2316), ('new', 2233), ('for', 1962), ('from', 1881), ('Wuhan', 1856)]
408123 total words


Some remarks-

* We can see that there are several article words (a, an , the), prepositions (in, of), as well as non-English prepositions (en, de) So we need to remove such non-essential words.
* 'coronavirus' and 'coronavirus.' (with a dot) are taken as two seperate words. So we may need to remove all such punctuation marks.
* 'Coronavirus' and 'coronavirus' are also treaded as two different words, so we need to take care of upper/lower case letters. 

In [79]:
# Convert all the letters of words to lowercase
word_list_lower = list(map(lambda x:x.lower(), word_list))

# Get the count values of all the words
words_counter = Counter(word_list_lower).most_common()

# Convert the Counter list to a Pandas dataframe
words_counter_df = pd.DataFrame.from_records(list(dict(words_counter).items()), columns=['word', 'count'])

print(len(word_list_lower), "total words")
print(len(Counter(word_list_lower)), "unique words")


with open('all-words.txt', 'w', encoding="utf-8") as filehandle:
    for listitem in word_list_lower:
        filehandle.write('%s\n' % listitem)

408123 total words
75228 unique words


In [80]:
import itertools

# Remove English, French and Spanish Parts of Speech (PoS) words
english_pos = ["a", "an", "the", "in", "on", "of", "for", "to", "by", "at", "till", "until", "i", "as", "it", "he", "she", "you", "via", "and"]
french_pos = ["en", "de", "le", "la", "les", "des", "à", "un", "une", "se"]
spanish_pos = ["el", "por", "que"]

for word1, word2, word3 in itertools.zip_longest(english_pos, french_pos, spanish_pos):
    words_counter_df.drop(words_counter_df[words_counter_df.word == word1].index, inplace=True)
    words_counter_df.drop(words_counter_df[words_counter_df.word == word2].index, inplace=True)
    words_counter_df.drop(words_counter_df[words_counter_df.word == word3].index, inplace=True)
    #print(word1, word2, word3)

words_counter_df.reset_index()
words_counter_df[:20]

,word,count
0,coronavirus,11232
7,#coronavirus,4237
8,china,4141
11,new,3169
15,is,2656
17,from,2034
18,wuhan,1892
19,-,1792
21,y,1530
23,that,1436


* We see that there are too many different types of PoS words that are coming across. It's better to utilize Python's NLTK library for further refinement.



